In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> ### 라이브러리 호출

In [2]:
import os
from pprint import pprint

# 데이터 전처리 및 모델링
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

#불필요한 경고 제거
import warnings
warnings.filterwarnings("ignore")

In [3]:
def read_excel_file(file_path: str, header: int = None) -> pd.DataFrame:
    csv_file = file_path.replace(".xlsx", ".csv")

    if not os.path.exists(csv_file):
        print("Converting excel to csv...")
        if header:
            df = pd.read_excel(file_path, header=header)
        else:
            df = pd.read_excel(file_path)

        df.to_csv(csv_file, index=False)
        print(f"  {file_path} -> {csv_file}")
        return df
    else:
        print(f"  Reading {csv_file}")
        return pd.read_csv(csv_file, low_memory=False)

In [4]:
# Columns 리스트 생략 없이 전부 출력하기
pd.set_option('display.max_seq_items', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [5]:
ROOT_DIR = "/content/drive/MyDrive/LG AImers/data set/"
RANDOM_STATE = 110

# Load_train_data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

In [ ]:
# Load_test_data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
submission_df = pd.read_csv(os.path.join(ROOT_DIR, "submission.csv"))

## 데이터 전처리

### STEP 1. Column의 값이 모두 Nan이거나 값이 모두 같은 feature 제거

In [ ]:
def remove_constant_columns(df: pd.DataFrame) -> pd.DataFrame:
  # 값이 모두 NaN인 열 제거
  df = df.dropna(axis=1, how='all')

  # NaN을 포함하여 unique한 값이 2개 이상인 열만 선택
  df = df.loc[:, df.apply(lambda x: x.nunique(dropna=False)) > 1]

  return df

### STEP 2. Solve Column Shift Problem

- HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result 값이 OK / NaN인 값은 모두 HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result 칼럼부터 WorkMode Collect Result까지 data 값이 하나씩 밀려있다.

> 1. split_dataframe_by_column_suffix => dam / fill1 / fill2 / autoclave 분리
2. shift_columns_forward_conditionally_dam => dam 관련 data 오류 해결
3. shift_columns_forward_conditionally_fill1  => fill1 관련 data 오류 해결
4. shift_columns_forward_conditionally_fill2 => fill2 관련 data 오류 해결

> **결측값 처리**
- 전체 40506개의 data 중 24059개의 결측값이 존재하기에 drop
- HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value & GMES_ORIGIN_INSP_JUDGE_CODE 관련 컬럼 DROP
- **WorkMode Collect Result_Dam, WorkMode Collect Result_Fill1, WorkMode Collect Result_Fill2 칼럼 DROP**

In [ ]:
# HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value 관련 컬럼 DROP
# GMES_ORIGIN_INSP_JUDGE_CODE 관련 컬럼 DROP

def drop_columns_with_string(df, substring):
  df_filtered = df.drop(columns=[col for col in df.columns if substring in col])
  return df_filtered

> 기존 dataframe에서 target column 추출

In [ ]:
train_data_target_df = train_data["target"]

In [ ]:
def split_dataframe_by_column_suffix(df):
    """
    주어진 데이터프레임에서 컬럼명이 'Dam', 'Fill1', 'Fill2', 'AutoClave'로 끝나는 열들로
    구성된 4개의 데이터프레임으로 나누어 반환합니다.

    Parameters:
    - df: pd.DataFrame - 원본 데이터프레임

    Returns:
    - pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame - 나눠진 4개의 데이터프레임
    """
    # 'Dam'으로 끝나는 칼럼
    dam_columns = [col for col in df.columns if col.endswith('Dam')]
    df_dam = df[dam_columns]

    # 'Fill1'로 끝나는 칼럼
    fill1_columns = [col for col in df.columns if col.endswith('Fill1')]
    df_fill1 = df[fill1_columns]

    # 'Fill2'로 끝나는 칼럼
    fill2_columns = [col for col in df.columns if col.endswith('Fill2')]
    df_fill2 = df[fill2_columns]

    # 'AutoClave'로 끝나는 칼럼
    autoclave_columns = [col for col in df.columns if col.endswith('AutoClave')]
    df_autoclave = df[autoclave_columns]

    return df_dam, df_fill1, df_fill2, df_autoclave

In [ ]:
import pandas as pd

def shift_columns_forward_conditionally_dam(df):
    # 복사본을 생성하여 원본을 유지합니다.
    df_shifted = df.copy()

    # 조건에 해당하는 행들을 필터링
    condition = df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].isin(["OK"]) | df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam"].isna()

    # 컬럼 이름 목록
    columns_to_shift = [
        "HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam",
        "HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam",
        "HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam",
        "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam",
        "HEAD Standby Position X Collect Result_Dam",
        "HEAD Standby Position Y Collect Result_Dam",
        "HEAD Standby Position Z Collect Result_Dam",
        "Head Clean Position X Collect Result_Dam",
        "Head Clean Position Y Collect Result_Dam",
        "Head Clean Position Z Collect Result_Dam",
        "Head Purge Position X Collect Result_Dam",
        "Head Purge Position Y Collect Result_Dam",
        "Head Purge Position Z Collect Result_Dam",
        "Head Zero Position X Collect Result_Dam",
        "Head Zero Position Y Collect Result_Dam",
        "Head Zero Position Z Collect Result_Dam",
        "Machine Tact time Collect Result_Dam",
        "PalletID Collect Result_Dam",
        "Production Qty Collect Result_Dam",
        "Receip No Collect Result_Dam",
        "Stage1 Circle1 Distance Speed Collect Result_Dam",
        "Stage1 Circle2 Distance Speed Collect Result_Dam",
        "Stage1 Circle3 Distance Speed Collect Result_Dam",
        "Stage1 Circle4 Distance Speed Collect Result_Dam",
        "Stage1 Line1 Distance Speed Collect Result_Dam",
        "Stage1 Line2 Distance Speed Collect Result_Dam",
        "Stage1 Line3 Distance Speed Collect Result_Dam",
        "Stage1 Line4 Distance Speed Collect Result_Dam",
        "Stage2 Circle1 Distance Speed Collect Result_Dam",
        "Stage2 Circle2 Distance Speed Collect Result_Dam",
        "Stage2 Circle3 Distance Speed Collect Result_Dam",
        "Stage2 Circle4 Distance Speed Collect Result_Dam",
        "Stage2 Line1 Distance Speed Collect Result_Dam",
        "Stage2 Line2 Distance Speed Collect Result_Dam",
        "Stage2 Line3 Distance Speed Collect Result_Dam",
        "Stage2 Line4 Distance Speed Collect Result_Dam",
        "Stage3 Circle1 Distance Speed Collect Result_Dam",
        "Stage3 Circle2 Distance Speed Collect Result_Dam",
        "Stage3 Circle3 Distance Speed Collect Result_Dam",
        "Stage3 Circle4 Distance Speed Collect Result_Dam",
        "Stage3 Line1 Distance Speed Collect Result_Dam",
        "Stage3 Line2 Distance Speed Collect Result_Dam",
        "Stage3 Line3 Distance Speed Collect Result_Dam",
        "Stage3 Line4 Distance Speed Collect Result_Dam",
        "THICKNESS 1 Collect Result_Dam",
        "THICKNESS 2 Collect Result_Dam",
        "THICKNESS 3 Collect Result_Dam",
        "WorkMode Collect Result_Dam"
    ]


    # 조건을 만족하는 행들에 대해서만 값을 이동
    for i in range(len(columns_to_shift) - 1):
        df_shifted.loc[condition, columns_to_shift[i]] = df_shifted.loc[condition, columns_to_shift[i + 1]]

    # 마지막 컬럼을 NaN으로 설정
    df_shifted.loc[condition, columns_to_shift[-1]] = pd.NA

    return df_shifted


In [ ]:
def shift_columns_forward_conditionally_fill1(df):
    # 복사본을 생성하여 원본을 유지합니다.
    df_shifted = df.copy()

    # 조건에 해당하는 행들을 필터링
    condition = df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].isin(["OK"]) | df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1"].isna()

    # 컬럼 이름 목록
    columns_to_shift = [
        "HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1",
        "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1",
        "HEAD Standby Position X Collect Result_Fill1",
        "HEAD Standby Position Y Collect Result_Fill1",
        "HEAD Standby Position Z Collect Result_Fill1",
        "Head Clean Position X Collect Result_Fill1",
        "Head Clean Position Y Collect Result_Fill1",
        "Head Clean Position Z Collect Result_Fill1",
        "Head Purge Position X Collect Result_Fill1",
        "Head Purge Position Y Collect Result_Fill1",
        "Head Purge Position Z Collect Result_Fill1",
        "Machine Tact time Collect Result_Fill1",
        "PalletID Collect Result_Fill1",
        "Production Qty Collect Result_Fill1",
        "Receip No Collect Result_Fill1",
        "WorkMode Collect Result_Fill1"
    ]

    # 조건을 만족하는 행들에 대해서만 값을 이동
    for i in range(len(columns_to_shift) - 1):
        df_shifted.loc[condition, columns_to_shift[i]] = df_shifted.loc[condition, columns_to_shift[i + 1]]

    # 마지막 컬럼을 pd.NA로 설정
    df_shifted.loc[condition, columns_to_shift[-1]] = pd.NA

    return df_shifted


In [ ]:
def shift_columns_forward_conditionally_fill2(df):
    # 복사본을 생성하여 원본을 유지합니다.
    df_shifted = df.copy()

    # 조건에 해당하는 행들을 필터링
    condition = df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].isin(["OK"]) | df["HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2"].isna()

    # 새로운 컬럼 이름 목록
    columns_to_shift = [
        "HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2",
        "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2",
        "HEAD Standby Position X Collect Result_Fill2",
        "HEAD Standby Position Y Collect Result_Fill2",
        "HEAD Standby Position Z Collect Result_Fill2",
        "Head Clean Position X Collect Result_Fill2",
        "Head Clean Position Y Collect Result_Fill2",
        "Head Clean Position Z Collect Result_Fill2",
        "Head Purge Position X Collect Result_Fill2",
        "Head Purge Position Y Collect Result_Fill2",
        "Head Purge Position Z Collect Result_Fill2",
        "Machine Tact time Collect Result_Fill2",
        "PalletID Collect Result_Fill2",
        "Production Qty Collect Result_Fill2",
        "Receip No Collect Result_Fill2",
        "WorkMode Collect Result_Fill2"
    ]

    # 조건을 만족하는 행들에 대해서만 값을 이동
    for i in range(len(columns_to_shift) - 1):
        df_shifted.loc[condition, columns_to_shift[i]] = df_shifted.loc[condition, columns_to_shift[i + 1]]

    # 마지막 컬럼을 pd.NA로 설정
    df_shifted.loc[condition, columns_to_shift[-1]] = pd.NA

    return df_shifted


### STEP 3. Data Frame 병합 후 WorkMode 관련 Column Drop

In [ ]:
def concatenate_dataframes(*dfs):
  # 데이터프레임의 개수가 4개 또는 5개인지 확인
  if len(dfs) not in [4, 5]:
    raise ValueError("입력 데이터프레임의 개수는 target을 포함한 5개이거나 target을 제외한 4개여야 합니다.")

  # 모든 데이터프레임이 동일한 행의 개수를 가지는지 확인
  row_counts = [df.shape[0] for df in dfs]
  if len(set(row_counts)) != 1:
    raise ValueError("모든 데이터프레임의 행의 개수가 동일해야 합니다.")

  # 데이터프레임들을 옆으로 붙이기
  merged_data_frame = pd.concat(dfs, axis=1)

  # 'WorkMode'가 컬럼명에 포함된 모든 컬럼을 드롭(drop)
  columns_to_drop = [col for col in merged_data_frame.columns if 'WorkMode' in col]
  merged_data_frame_drop_workmode_col = merged_data_frame.drop(columns = columns_to_drop)

  return merged_data_frame_drop_workmode_col

> **'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result' 관련 컬럼의 데이터 타입 숫자형으로 변경**
- Column Shift 문제를 해결한 후 HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result 관련 컬럼들의 data type이 모두 category형으로 남아있기에 float 타입으로 변경해준다.

In [ ]:
def convert_columns_to_float(df, keyword):
    """
    Parameters:
    - df (pd.DataFrame): 변환할 데이터 프레임
    - keyword (str): 칼럼명에 포함된 키워드

    Returns:
    - pd.DataFrame: 변환된 데이터 프레임
    """
    # 키워드를 포함하는 칼럼명을 찾기
    columns_to_convert = [col for col in df.columns if keyword in col]

    # 해당 칼럼들을 실수형으로 변환
    df[columns_to_convert] = df[columns_to_convert].astype(float)

    return df

### STEP 4

#### STEP 4-1. 제조 공정 중 Qty 변화 있으면 AbNormal

In [ ]:
def filter_different_rows_Qty(df):
    # 제조 공정 중 Qty 칼럼값에 변화가 있는 row
    condition = (df['Production Qty Collect Result_Dam'] != df['Production Qty Collect Result_Fill1']) | \
                (df['Production Qty Collect Result_Dam'] != df['Production Qty Collect Result_Fill2']) | \
                (df['Production Qty Collect Result_Fill1'] != df['Production Qty Collect Result_Fill2'])

    # 조건을 만족하는 행들로 구성된 데이터프레임
    filtered_df = df[condition]

    # 입력 데이터프레임과 필터링된 데이터프레임 반환
    return filtered_df, df

#### STEP 4-2. 제조 공정 중 Pallet ID 변화 있으면 AbNormal

In [ ]:
def filter_different_rows_PalletID(df):
    # 제조 공정 중 Pallet ID 칼럼값에 변화가 있는 row
    condition = (df['PalletID Collect Result_Dam'] != df['PalletID Collect Result_Fill1']) | \
                (df['PalletID Collect Result_Dam'] != df['PalletID Collect Result_Fill2']) | \
                (df['PalletID Collect Result_Fill1'] != df['PalletID Collect Result_Fill2'])

    # 조건을 만족하는 행들로 구성된 데이터프레임
    filtered_df = df[condition]

    # 입력 데이터프레임과 필터링된 데이터프레임 반환
    return filtered_df, df

#### STEP 4-3. 제조 공정 중 Receip No 변화 있으면 AbNormal

In [ ]:
def filter_different_rows_Receip(df):
    # 제조 공정 중 Receip No 칼럼값에 변화가 있는 row
    condition = (df['Receip No Collect Result_Dam'] != df['Receip No Collect Result_Fill1']) | \
                (df['Receip No Collect Result_Dam'] != df['Receip No Collect Result_Fill2']) | \
                (df['Receip No Collect Result_Fill1'] != df['Receip No Collect Result_Fill2'])

    # 조건을 만족하는 행들로 구성된 데이터프레임
    filtered_df = df[condition]

    # 입력 데이터프레임과 필터링된 데이터프레임 반환
    return filtered_df, df

#### STEP 4-4. 제조 공정 중 Equipment 변화 있으면 AbNormal

In [ ]:
def filter_different_rows_Equipment(df):
    # 각 칼럼의 값의 마지막 글자 추출
    last_char_dam = df['Equipment_Dam'].astype(str).str[-1]
    last_char_fill1 = df['Equipment_Fill1'].astype(str).str[-1]
    last_char_fill2 = df['Equipment_Fill2'].astype(str).str[-1]

    # 제조 공정 중 Equipment 칼럼값에 변화가 있는 row
    condition = (last_char_dam != last_char_fill1) | (last_char_dam != last_char_fill2) | (last_char_fill1 != last_char_fill2)

    # 조건을 만족하는 행들로 구성된 데이터프레임
    filtered_df = df[condition]

    # 입력 데이터프레임과 필터링된 데이터프레임 반환
    return filtered_df, df

### STEP 5. 무조건 AbNormal인 값들의 index 저장하기

> 무조건 AbNormal인 row의 index들을 저장한 후, 학습 과정에서는 제외시킨 후 이후에 최종 과정에서 바꿔준다.

In [ ]:
# 무조건 Error가 발생하는 데이터들의 row index를 반환하는 함수
def get_unique_row_indices_with_AbNormal(*dfs):
    """
    dfs (tuple): 데이터 프레임들의 튜플
    ex) AbNormal_row_index = get_unique_row_indices(data_with_dispensesr_error_df, data_with_Qty_error_df)
    """
    # 중복을 제거하기 위해 set 사용
    unique_indices = set()

    # 각 데이터 프레임에 대해 인덱스를 추출하여 set에 추가
    for df in dfs:
      unique_indices.update(df.index)

    # 중복 없는 인덱스를 리스트로 변환하여 반환
    return list(unique_indices)

> **Change in the Column 파생변수 생성**

In [ ]:
def add_change_column(df, AbNormal_row_index_list):
  df['Change in the Column'] = df.index.isin(AbNormal_row_index_list).astype(int)
  return df

### STEP 6. Dispenser의 종류에 따라 Data Frame Split

In [ ]:
# Equipment_Dam를 각 제품들의 original 설정값이라는 가정하에, dam에서의 dispenser에 따라 데이터프레임 분할
def Seperate_data_by_Dam_dispenser(df):
  # Equipment_Dam 칼럼의 값이 "Dam dispenser #1"인 경우
  df_dispenser_1 = df[df['Equipment_Dam'] == 'Dam dispenser #1']

  # Equipment_Dam 칼럼의 값이 "Dam dispenser #2"인 경우
  df_dispenser_2 = df[df['Equipment_Dam'] == 'Dam dispenser #2']

  # 각각의 분할된 데이터프레임 반환
  return df_dispenser_1, df_dispenser_2

> **Dispenser에 따라 나눈 후, Equipment 관련 컬럼 DROP**

```python
dispenser1_df_train = drop_columns_with_string(dispenser1_df_train, "Equipment")
dispenser2_df_train = drop_columns_with_string(dispenser2_df_train, "Equipment")
```



### STEP 7. Dispenser 구별 후 행이 같은 경우에 중복되는 값을 가지는 Columns DROP

In [ ]:
def find_identical_value_columns(df):
  # 결과를 저장할 딕셔너리 생성
  identical_columns = {}

  # 데이터 프레임의 모든 칼럼을 순회
  for i, col1 in enumerate(df.columns):
    for col2 in df.columns[i+1:]:
      # 두 칼럼의 값이 동일한지 확인
      if df[col1].equals(df[col2]):
        # 동일한 값을 가지는 칼럼을 리스트에 추가
        if col1 not in identical_columns:
          identical_columns[col1] = []
          identical_columns[col1].append(col2)

  return identical_columns

In [ ]:
from collections import defaultdict

def build_graph(identical_columns):
    graph = defaultdict(set)

    for key, values in identical_columns.items():
        for value in values:
            graph[key].add(value)
            graph[value].add(key)

    return graph

def find_connected_components(graph):
    visited = set()
    components = []

    def dfs(node, component):
        stack = [node]
        while stack:
            v = stack.pop()
            if v not in visited:
                visited.add(v)
                component.append(v)
                stack.extend(graph[v] - visited)

    for node in graph:
        if node not in visited:
            component = []
            dfs(node, component)
            components.append(component)

    return components

In [ ]:
def drop_duplicate_columns(df, components):
  # Collect columns to drop
  columns_to_drop = set()
  for component in components:
    if len(component) > 1:
      # Skip the first column, mark others for dropping
      columns_to_drop.update(component[1:])

  # Drop columns from the DataFrame
  final_df = df.drop(columns=columns_to_drop, errors='ignore')

  return final_df

### STEP 8. Dispenser 구별 후 UNIQUE한 값을 1개만 가지는 Columns DROP

In [ ]:
# 주어진 데이터프레임에서 고유한 값이 1개인 칼럼을 제거하고 새로운 데이터프레임을 반환
def drop_single_unique_columns(df):
  # 각 칼럼의 고유 값 개수를 확인
  unique_counts = df.nunique()

  # 고유 값이 1개인 칼럼 이름 찾기
  single_unique_cols = unique_counts[unique_counts == 1].index

  # 해당 칼럼을 제거한 새로운 데이터프레임 생성
  df_dropped = df.drop(columns=single_unique_cols)
  return df_dropped

### STEP 9. Target Column 및 Object 데이터 타입 Column들 데이터 타입 변경
- Normal => 0
- AbNormal => 1
- 칼럼의 데이터 타입을 정수형으로 변환

In [ ]:
# Object data type => category형으로 변환
def convert_to_category(df):
  """
  주어진 데이터 프레임에서 target 칼럼을 제외한 수치형이 아닌 모든 칼럼에 대해 범주형으로 변환
  Returns : pandas.DataFrame: 범주형으로 변환된 데이터 프레임
  """
  # 'target' 칼럼이 있는 경우, 이를 제외한 나머지 칼럼들을 처리
  if 'target' in df.columns:
    columns_to_convert = df.select_dtypes(exclude=['number']).columns.difference(['target'])
  else:
    columns_to_convert = df.select_dtypes(exclude=['number']).columns

  # 수치형이 아닌 칼럼들을 범주형으로 변환
  df[columns_to_convert] = df[columns_to_convert].astype('category')

  return df

In [ ]:
# target값 0과 1로 변환
def change_target_column_data_type(df):
  df["target"] = df["target"].map({"Normal": 0, "AbNormal": 1})
  df["target"] = df["target"].astype(int)
  return df

## Train Data

In [ ]:
# Column의 값이 모두 Nan이거나 값이 모두 같은 feature 제거
train_data = remove_constant_columns(train_data)

# HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value 관련 컬럼 DROP
# GMES_ORIGIN_INSP_JUDGE_CODE 관련 컬럼 DROP
train_data = drop_columns_with_string(train_data, "HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value")
train_data = drop_columns_with_string(train_data, "GMES_ORIGIN_INSP_JUDGE_CODE")

# 기존 dataframe에서 target column 추출
train_data_target_df = train_data["target"]

#  Solve Column Shift Problem
df_dam, df_fill1, df_fill2, df_autoclave = split_dataframe_by_column_suffix(train_data)
df_dam_correct = shift_columns_forward_conditionally_dam(df_dam)
df_fill1_correct = shift_columns_forward_conditionally_fill1(df_fill1)
df_fill2_correct = shift_columns_forward_conditionally_fill2(df_fill2)

# dataframe 병합
train_data_correct = concatenate_dataframes(df_dam_correct, df_fill1_correct, df_fill2_correct, df_autoclave, train_data_target_df)

# 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result' 관련 컬럼의 데이터 타입 숫자형으로 변경
train_data_correct = convert_columns_to_float(train_data_correct, 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result')

#############무조건 AbNormal인 row의 index 저장###############
# 제조 공정 중 Qty 변화가 있는 데이터 프레임
data_with_Qty_error_df_train, train_data_correct = filter_different_rows_Qty(train_data_correct)

# 제조 공정 중 PalletID 변화 있는 데이터 프레임
data_with_PalletID_error_df_train, train_data_correct = filter_different_rows_PalletID(train_data_correct)

# 제조 공정 중 Receip No 변화 있는 데이터 프레임
data_with_Receip_error_df_train, train_data_correct = filter_different_rows_Receip(train_data_correct)

# 제조 공정 중 Equipment 변화 있는 데이터 프레임
data_with_Equipment_error_df_train, train_data_correct = filter_different_rows_Equipment(train_data_correct)

# 무조건 AbNormal인 값들의 index 저장하기
AbNormal_row_index_list_train = get_unique_row_indices_with_AbNormal(data_with_Qty_error_df_train, data_with_PalletID_error_df_train, data_with_Receip_error_df_train, data_with_Equipment_error_df_train)

# 값에 변화가 있는 row들을 구별해주는 Change in the Column 파생변수 생성
train_data_correct = add_change_column(train_data_correct, AbNormal_row_index_list_train)

# Dispenser의 종류에 따라 Data Frame Split
dispenser1_df_train , dispenser2_df_train = Seperate_data_by_Dam_dispenser(train_data_correct)

# Equipment 관련 컬럼 DROP
dispenser1_df_train = drop_columns_with_string(dispenser1_df_train, "Equipment")
dispenser2_df_train = drop_columns_with_string(dispenser2_df_train, "Equipment")

# 행이 같은 경우에 중복되는 값을 가지는 Columns DROP
identical_columns_in_df1_train = find_identical_value_columns(dispenser1_df_train)
identical_columns_in_df2_train = find_identical_value_columns(dispenser2_df_train)
graph1_train = build_graph(identical_columns_in_df1_train)
graph2_train = build_graph(identical_columns_in_df2_train)
components1_train = find_connected_components(graph1_train)
components2_train = find_connected_components(graph2_train)
dispenser1_df_train = drop_duplicate_columns(dispenser1_df_train, components1_train)
dispenser2_df_train = drop_duplicate_columns(dispenser2_df_train, components2_train)

# UNIQUE한 값을 가지는 Columns DROP
dispenser1_df_train = drop_single_unique_columns(dispenser1_df_train)
dispenser2_df_train = drop_single_unique_columns(dispenser2_df_train)

# Model.Suffix, Workorder 관련 칼럼 DROP
dispenser1_df_train = drop_columns_with_string(dispenser1_df_train, "Model.Suffix")
dispenser1_df_train = drop_columns_with_string(dispenser1_df_train, "Workorder")
dispenser2_df_train = drop_columns_with_string(dispenser2_df_train, "Model.Suffix")
dispenser2_df_train = drop_columns_with_string(dispenser2_df_train, "Workorder")

# Object data type => category형으로 변환
dispenser1_df_train = convert_to_category(dispenser1_df_train)
dispenser2_df_train = convert_to_category(dispenser2_df_train)

# target column -> 0(Normal)과 1(AbNormal)로 변경
dispenser1_df_train = change_target_column_data_type(dispenser1_df_train)
dispenser2_df_train = change_target_column_data_type(dispenser2_df_train)

## Test Data

In [ ]:
def filter_columns_for_test_dataset_first(test_data_df, train_data_df):
  dataframe_for_Columns = remove_constant_columns(train_data)
  dataframe_for_Columns = drop_columns_with_string(dataframe_for_Columns, "HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value")
  dataframe_for_Columns = drop_columns_with_string(dataframe_for_Columns, "GMES_ORIGIN_INSP_JUDGE_CODE")

  # train dataset의 컬럼들
  first_remain_column_list = dataframe_for_Columns.columns

  # 입력 데이터프레임의 칼럼 목록
  df_columns = test_data_df.columns

  # 데이터프레임의 칼럼 중 train_data에 있는 칼럼만 필터링
  columns_to_keep = [col for col in df_columns if col in first_remain_column_list]
  filtered_df = test_data_df[columns_to_keep]
  return filtered_df

In [ ]:
def filter_test_columns(dispenser_df_test, dispenser_df_train):
    """ dispenser_df_test 데이터프레임에서 dispenser1_df_train 데이터프레임에 없는 칼럼을 제거하고 새로운 데이터프레임을 반환"""
    # 훈련 데이터프레임의 칼럼 목록
    train_columns = dispenser_df_train.columns
    # 테스트 데이터프레임의 칼럼 목록
    test_columns = dispenser_df_test.columns

    # 훈련 데이터프레임에 존재하는 칼럼만 테스트 데이터프레임에서 선택합니다
    columns_to_keep = [col for col in test_columns if col in train_columns]
    # 선택한 칼럼만 포함된 새로운 데이터프레임을 생성합니다
    filtered_test_df = dispenser_df_test[columns_to_keep]

    return filtered_test_df

In [ ]:
# Column shifting problem 해결 전 feauture 통일
test_data = filter_columns_for_test_dataset_first(test_data, train_data)

#  Solve Column Shift Problem
df_dam_test, df_fill1_test, df_fill2_test, df_autoclave_test = split_dataframe_by_column_suffix(test_data)
df_dam_correct_test = shift_columns_forward_conditionally_dam(df_dam_test)
df_fill1_correct_test = shift_columns_forward_conditionally_fill1(df_fill1_test)
df_fill2_correct_test = shift_columns_forward_conditionally_fill2(df_fill2_test)

# dataframe 병합
test_data_correct = concatenate_dataframes(df_dam_correct_test, df_fill1_correct_test, df_fill2_correct_test, df_autoclave_test)

# 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result' 관련 컬럼의 데이터 타입 숫자형으로 변경
test_data_correct = convert_columns_to_float(test_data_correct, 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result')


###########무조건 AbNormal인 row의 index 저장############
data_with_Qty_error_df_test, test_data_correct = filter_different_rows_Qty(test_data_correct)

# 제조 공정 중 PalletID 변화 있는 데이터 프레임
data_with_PalletID_error_df_test, test_data_correct = filter_different_rows_PalletID(test_data_correct)

# 제조 공정 중 Receip No 변화 있는 데이터 프레임
data_with_Receip_error_df_test, test_data_correct = filter_different_rows_Receip(test_data_correct)

# 제조 공정 중 Equipment 변화 있는 데이터 프레임
data_with_Equipment_error_df_test, test_data_correct = filter_different_rows_Equipment(test_data_correct)

# 무조건 AbNormal인 값들의 index 저장하기
AbNormal_row_index_list_test = get_unique_row_indices_with_AbNormal(data_with_Qty_error_df_test, data_with_PalletID_error_df_test, data_with_Receip_error_df_test, data_with_Equipment_error_df_test)

# 값에 변화가 있는 row들을 구별해주는 Change in the Column 파생변수 생성
test_data_correct = add_change_column(test_data_correct, AbNormal_row_index_list_test)

# Dispenser의 종류에 따라 Data Frame Split
dispenser1_df_test, dispenser2_df_test = Seperate_data_by_Dam_dispenser(test_data_correct)

# train data와 feature 맞춰주기
dispenser1_df_test = filter_test_columns(dispenser1_df_test, dispenser1_df_train)
dispenser2_df_test = filter_test_columns(dispenser2_df_test, dispenser2_df_train)

# Object data type => category형으로 변환
dispenser1_df_test = convert_to_category(dispenser1_df_test)
dispenser2_df_test = convert_to_category(dispenser2_df_test)

단순히 Automl

# 발표 듣고

- **discharged time of resin : 0인것. => 무조건 abnormal**

- **Thickness에 따라 volume이 중요할거 같은데??**

=> **칼럼 분포에서 이 값이 차지하는 비율**

==============================
- 1. catboost든 1svm이든 예측해서 날리고
- 2. auto encoder로 autoclave는 남겨두고 dam, fill1, fill2 차원 축소 시키고
- 3. smote로 데이터 늘려서
- 예측하면 될듯
- 2,3의 순서는 고민을 해봐야 할듯

- pressure time이 230 under이면 불량일 확률이 높아보인다
- OK : 파랑 , NG : 빨강 , 값 : Collect Result * Unit Time

## 해볼 것

1. 단순 하이퍼 파라미터 튜닝으로 성능 올리기
- 모델 선택 : pycaret
- 하이퍼 파라미터 튜닝 : optuna

2. 확실히 정상인 것을 제외하고

3. SMOTE로 늘리기 => 오토인코더 => 시도

4. 오토 인코더 => SMOTE로 늘리기 => 시도

https://baechu-story.tistory.com/68

gradient boosting or xgboost(scale_pos_weight 매개변수, 고차원 data, 수치형 data)를 써보는 게 좋을 것 같다. category형도 없으니까!

+
다른 분류 알고리즘들도 성능을 test 해보는 것으로!

permutation_importances_mean => n_repeats 매개변수로 feature를 10번정도 섞어서 확인을 해본다. => 모델이 어떤 특성을 중심으로 보는 지 알 수 있다.

Optuna stratified Kfold

## TRAIN MODEL

In [6]:
ROOT_DIR = "/content/drive/MyDrive/LG AImers/data set/Final Data/"
RANDOM_STATE = 110
# Load_train_data_dispenser1
dispenser1_df_train = pd.read_csv(os.path.join(ROOT_DIR, "dispenser1_df_train.csv"),index_col=0)
dispenser2_df_train = pd.read_csv(os.path.join(ROOT_DIR, "dispenser2_df_train.csv"),index_col=0)
dispenser1_df_test = pd.read_csv(os.path.join(ROOT_DIR, "dispenser1_df_test.csv"),index_col=0)
dispenser2_df_test = pd.read_csv(os.path.join(ROOT_DIR, "dispenser2_df_test.csv"),index_col=0)
submission_df = pd.read_csv(os.path.join("/content/drive/MyDrive/LG AImers/data set/submission.csv"))

In [7]:
# dataframe에서 'target'을 기준으로 stratify를 설정하여 데이터 분포를 유지하면서 train/validation split 수행
def model_train_test_split(dataframe):
  train_x, val_x, train_y, val_y = train_test_split(
  dataframe.drop(columns=["target"]),  # feature 데이터
  dataframe["target"],                 # target 데이터
  test_size=0.3,                      # 검증 데이터 비율
  random_state=RANDOM_STATE,          # 랜덤 시드
  stratify = dataframe["target"])         # target 값을 기준으로 stratify

  return train_x, train_y, val_x, val_y

In [9]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00


In [10]:
from sklearn.model_selection import KFold,GridSearchCV,train_test_split,StratifiedKFold
from sklearn.metrics import log_loss,f1_score
# train validation split
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

def train_catboost_with_gridsearch(dispenser1_df_train, K=5):
    df = dispenser1_df_train
    df['Chamber Temp. Judge Value_AutoClave'] = df['Chamber Temp. Judge Value_AutoClave'].map({'OK': 1, 'NG': 0})
    X_train = df.drop(columns='target')
    y_train = df['target']

    # Define the CatBoostClassifier
    model = CatBoostClassifier(
        task_type='GPU',          # Use GPU for training
        silent=True
    )

    # Define the expanded parameter grid for GridSearchCV
    param_grid = {
        'depth': [4, 6, 8, 10, 12],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'iterations': [500, 1000, 1500, 2000],
        'l2_leaf_reg': [1, 3, 5, 7, 10],
        'min_child_samples': [5, 10, 20]
    }

    # Define the recall scorer
    recall_scorer = make_scorer(recall_score)

    # Set up GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=recall_scorer,
        cv=K,
        n_jobs=-1
    )

    # Perform grid search
    grid_search.fit(X_train, y_train)

    # Return the best model
    return grid_search.best_estimator_

# Example usage
best_model = train_catboost_with_gridsearch(dispenser1_df_train, K=5)

print("Best model:", best_model)

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

def train_catboost_with_gridsearch(dispenser1_df_train, K=5):
    df = dispenser1_df_train
    df['Chamber Temp. Judge Value_AutoClave'] = df['Chamber Temp. Judge Value_AutoClave'].map({'OK': 1, 'NG': 0})
    X_train = df.drop(columns='target')
    y_train = df['target']

    # Define the CatBoostClassifier
    model = CatBoostClassifier(
        task_type='GPU',          # Use GPU for training
        silent=True
    )

    # Define the expanded parameter grid for GridSearchCV
    param_grid = {
        'depth': [4, 6, 8, 10, 12],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'iterations': [500, 1000, 1500, 2000],
        'l2_leaf_reg': [1, 3, 5, 7, 10],
        'min_child_samples': [5, 10, 20]
    }

    # Define the recall scorer
    recall_scorer = make_scorer(recall_score)

    # Set up GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=recall_scorer,
        cv=K,
        n_jobs=-1
    )

    # Perform grid search
    grid_search.fit(X_train, y_train)

    # Return the best model
    return grid_search.best_estimator_

# Example usage
best_model2 = train_catboost_with_gridsearch(dispenser2_df_train, K=5)

print("Best model:", best2_model)

> 모델 훈련 시작

In [ ]:
# dispenser 1
cat_model_dispenser1 = cat_model_fit(train_x_dispenser1, train_y_dispenser1, val_x_dispenser1, val_y_dispenser1)

0:	learn: 0.6319386	test: 0.6321164	best: 0.6321164 (0)	total: 268ms	remaining: 8m 56s
100:	learn: 0.2013061	test: 0.2129602	best: 0.2129563 (98)	total: 13.6s	remaining: 4m 14s
200:	learn: 0.1868867	test: 0.2116753	best: 0.2116753 (200)	total: 26.9s	remaining: 4m
300:	learn: 0.1718218	test: 0.2116512	best: 0.2114121 (247)	total: 44.8s	remaining: 4m 13s
400:	learn: 0.1548431	test: 0.2131854	best: 0.2114121 (247)	total: 1m 12s	remaining: 4m 47s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.2114120773
bestIteration = 247

Shrink model to first 248 iterations.


In [ ]:
# dispenser 2
cat_model_dispenser2 = cat_model_fit(train_x_dispenser2, train_y_dispenser2, val_x_dispenser2, val_y_dispenser2)

0:	learn: 0.6320790	test: 0.6320693	best: 0.6320693 (0)	total: 63.7ms	remaining: 2m 7s
100:	learn: 0.1747951	test: 0.1935157	best: 0.1934720 (97)	total: 10.2s	remaining: 3m 10s
200:	learn: 0.1506364	test: 0.1931279	best: 0.1927627 (132)	total: 22.2s	remaining: 3m 18s
300:	learn: 0.1303781	test: 0.1942097	best: 0.1927627 (132)	total: 34.7s	remaining: 3m 15s
Stopped by overfitting detector  (250 iterations wait)

bestTest = 0.192762664
bestIteration = 132

Shrink model to first 133 iterations.


> **여기서부터는 prediction threshold, model parameter 조정해보는거야**

In [ ]:
# 예측 확률에 AbNormal로 뺴두었던 index 추가하기

def add_unconditionally_AbNormal_dispenser1(pred_proba_numpy_array, val_x, submission_abnormal_row_index):
  # y_pred_proba1을 DataFrame으로 변환
  y_pred_proba_df = pd.DataFrame(y_pred_proba1, index=val_x_dispenser1.index, columns=['probability'])

  # Submission_AbNormal_row_index의 인덱스를 y_pred_proba_df에 추가하고, 값을 1로 설정
  for idx in submission_abnormal_row_index:
    if idx not in y_pred_proba_df.index:
      y_pred_proba_df.loc[idx] = 1.0  # 인덱스가 없으면 추가하고 값은 1.0으로 설정

  # y_pred_proba_df을 다시 Numpy로 변경
  modified_pred_proba_array = y_pred_proba_df.sort_index()['probability'].values

  return modified_pred_proba_array

In [ ]:
# validatoin_y에 AbNormal 추가하기
def add_abnormal_rows(val_y, submission_abnormal_row_index):
  # val_y의 기존 인덱스
  existing_indices = val_y.index.tolist()
  # Submission_AbNormal_row_index에서 val_y의 인덱스와 겹치지 않는 인덱스를 필터링
  new_indices = [idx for idx in submission_abnormal_row_index if idx not in existing_indices]

  # 기존 데이터프레임에 새로운 인덱스를 추가하고 'target' 값을 1로 설정
  for idx in new_indices:
    val_y.loc[idx] = 1
  # 인덱스를 정렬
  updated_val_y = val_y.sort_index()

  # 인덱스를 정렬
  updated_val_y = updated_val_y.values

  return updated_val_y

In [ ]:
# 예측 - dispenser1
y_pred_proba1 = cat_model_dispenser1.predict_proba(val_x_dispenser1)[:, 1]
y_pred_proba1 = add_unconditionally_AbNormal_dispenser1(y_pred_proba1, val_x_dispenser1, Submission_AbNormal_row_index)
y_pred1 = (y_pred_proba1 > 0.3).astype(int)
updated_val_y = add_abnormal_rows(val_y_dispenser1, Submission_AbNormal_row_index)
# F1 스코어 계산
f1 = f1_score(updated_val_y, y_pred1)
print(f'F1 Score: {f1:.4f}')

F1 Score: 0.3449


In [ ]:
# 예측 확률에 AbNormal로 뺴두었던 index 추가하기

def add_unconditionally_AbNormal_dispenser2(pred_proba_numpy_array, val_x, submission_abnormal_row_index):
  # y_pred_proba1을 DataFrame으로 변환
  y_pred_proba_df = pd.DataFrame(y_pred_proba2, index=val_x_dispenser2.index, columns=['probability'])

  # Submission_AbNormal_row_index의 인덱스를 y_pred_proba_df에 추가하고, 값을 1로 설정
  for idx in submission_abnormal_row_index:
    if idx not in y_pred_proba_df.index:
      y_pred_proba_df.loc[idx] = 1.0  # 인덱스가 없으면 추가하고 값은 1.0으로 설정

  # y_pred_proba_df을 다시 Numpy로 변경
  modified_pred_proba_array = y_pred_proba_df.sort_index()['probability'].values

  return modified_pred_proba_array

In [ ]:
# 예측 - dispenser2
y_pred_proba2 = cat_model_dispenser2.predict_proba(val_x_dispenser2)[:, 1]
y_pred_proba2 = add_unconditionally_AbNormal_dispenser2(y_pred_proba2, val_x_dispenser2, Submission_AbNormal_row_index)
y_pred2 = (y_pred_proba2 > 0.3).astype(int)
updated_val_y_2 = add_abnormal_rows(val_y_dispenser2, Submission_AbNormal_row_index)
# F1 스코어 계산
f2 = f1_score(updated_val_y_2, y_pred2)
print(f'F1 Score: {f2:.4f}')

F1 Score: 0.4989


### Analizing Prediction Result_TRAIN

In [ ]:
train_x_dispenser1 = convert_to_category(train_x_dispenser1)

# 특성 중요도 계산 및 출력
category_columns = train_x_dispenser1.select_dtypes(include='category').columns.tolist()
feature_importances = cat_model_dispenser1.get_feature_importance(Pool(train_x_dispenser1, label=train_y_dispenser1, cat_features=category_columns))

# 특성 중요도를 내림차순으로 정렬하고 상위 30개의 인덱스를 선택
sorted_idx = np.argsort(feature_importances)[::-1]  # 내림차순으로 정렬된 인덱스
top_40_idx = sorted_idx[:40]  # 상위 30개의 인덱스 선택

# 상위 30개의 특성과 중요도만 선택
top_40_features = train_x_dispenser1.columns[top_40_idx]
top_40_importances = feature_importances[top_40_idx]

# 상위 30개 특성에 대해 시각화
plt.figure(figsize=(10, 10))
plt.barh(top_40_features, top_40_importances)
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Top 40 Feature Importance in CatBoost Model')
plt.show()

### PREDICT & SCORE => Test data

In [ ]:
dispenser1_df_test.shape[0]

10731

In [ ]:
dispenser2_df_test.shape[0]

6618

In [ ]:
y_pred_proba1_test = cat_model_dispenser1.predict_proba(dispenser1_df_test)[:, 1]
y_pred1_test = (y_pred_proba1_test > 0.11).astype(int)

y_pred_proba2_test = cat_model_dispenser2.predict_proba(dispenser2_df_test)[:, 1]
y_pred2_test = (y_pred_proba2_test > 0.11).astype(int)

# submission에 대입
submission_df.loc[dispenser1_df_test.index, 'target'] = y_pred1_test
submission_df.loc[dispenser2_df_test.index, 'target'] = y_pred2_test
submission_df['target'] = submission_df['target'].map({0: 'Normal', 1: 'AbNormal'})
submission_df.loc[Submission_AbNormal_row_index_test, 'target'] = "AbNormal"

In [ ]:
# predict 임계값 : 0.11, depth : 5 , earlystopping : 40  ==> 0.185
submission_df["target"].value_counts()
# 15:1

,count
target,
Normal,16276
AbNormal,1085


In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
train_data["target"].value_counts()
# 16 : 1

,count
target,
Normal,38156
AbNormal,2350


> #### 데이터를 고쳤는데도 점수가 거의 오르지 않은 것으로 보아, 확실히 AbNormal인 데이터들이 적은데 빼니까 더 감지가 어려운듯함. 처음부터 제외하고 학습을 시키기 보다는 같이 학습을 시키는 것이 좋아보임

### 1. undersampling

# 여기부터는 생각 더!

### STEP 특징을 살펴보고 unique한 값이 오직 하나면 drop할 지 결정하자.

- CURE START POSITION Θ Collect Result_Dam == CURE END POSITION Θ Collect Result_Dam
- 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam' => stage 1,2,3가 동일
- HeadNormal Y_dam도 stage 1,3가 3개 제외하고는 다 같은 듯

> **조금 특이**
- 'HEAD Standby Position X Collect Result_Dam' ==  ['Head Purge Position X Collect Result_Dam'],
- 'HEAD Standby Position Y Collect Result_Dam' ==  ['Head Purge Position Y Collect Result_Dam'],
- **'HEAD Standby Position Z Collect Result_Dam' !=  ['Head Purge Position Z Collect Result_Dam'],**
- 'HEAD Standby Position Z Collect Result_Dam' == ['HEAD Standby Position Z Collect Result_Fill1'], => **얘는 다른 이유가 뭘까**

### Column의 unique한 값이 1개뿐인 column

### STEP 5. 모든 공정에서 값이 동일한 column drop ==> 딱 1개만 남겨두는 방향으로!
- Model suffix
- workorder

> 데이터를 다시 해보니, 값이 아예 동일한 컬럼들이 존재하네?

## g

- work order의 1234는 의미가 있는 숫자일까?
- chamber temp 가 중간인 곳과 아닌 곳으로 chamber temp가 NG OK 구분됨
- Machine tact time이 정상 비정상에서 차이가 크다
- Qty가 0은 무슨 의미이지?
- 1st Pressure 1st Pressure Unit Time_AutoClave 가 0인 값들이 있네? 1st & 2nd
- 이때 1st, 2nd가 둘 다 time이 0인 값은 없어
- 3rd는 무조건 1초이상인데!


>- 압력의 범위가 Normal일 때가 더 넓어
- **Abnormal 일때는 더 넓어야 하는데, 그러지 못해서 그랬을 확률이 있어보여.**


> Dam dispenser에 따라 CURE START & END POSITION & HEAD NORMAL COORDINATE 가 달라진
- head는 분명한 순서가 존재하는  과정

- model.SuffixDam은 디스플레이 모델을 구분하는 것이라면... 이게 dam 높이라던지 레진 도포 위치 등의 공정의 좌표를 결정하지 않을까?